# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* **PEFT technique:** LoRA
* **Model:** `DeBERTa` improves the BERT and RoBERTa models using disentangled attention and enhanced mask decoder. With those two improvements, DeBERTa out perform RoBERTa on a majority of NLU tasks with 80GB training data.

More info at : https://huggingface.co/microsoft/deberta-v3-base

* **Evaluation approach:**
* **Fine-tuning dataset:** `lmsys/toxic-chat` -> This dataset contains toxicity annotations on 10K user prompts collected from the Vicuna online demo.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
# please first run this cell
!pip install --upgrade huggingface_hub datasets

In [1]:
from datasets import load_dataset_builder, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# load the data set
ds = load_dataset("lmsys/toxic-chat", "toxicchat0124")
ds = ds.rename_column("toxicity", "labels")
ds

README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'user_input', 'model_output', 'human_annotation', 'labels', 'jailbreaking', 'openai_moderation'],
        num_rows: 5082
    })
    test: Dataset({
        features: ['conv_id', 'user_input', 'model_output', 'human_annotation', 'labels', 'jailbreaking', 'openai_moderation'],
        num_rows: 5083
    })
})

In [3]:
columns_to_keep = ["user_input", "labels"]  # Las columnas que quieres conservar
ds = ds.remove_columns([col for col in ds["train"].column_names if col not in columns_to_keep])
ds

DatasetDict({
    train: Dataset({
        features: ['user_input', 'labels'],
        num_rows: 5082
    })
    test: Dataset({
        features: ['user_input', 'labels'],
        num_rows: 5083
    })
})

### Import DeBerta v3

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples."""
    return tokenizer(examples["user_input"], truncation=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else '[PAD]'
    tokenizer.add_special_tokens({'pad_token': tokenizer.pad_token})
    
# tokenize the inputs
train_tokenized = ds["train"].map(preprocess_function, batched=True)
test_tokenized = ds["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

Because we want to evaluate the base model, we will first freeze its parameters

In [6]:
# Freeze all the parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

print(model)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [7]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}



### Train the model

In [8]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer_base = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/toxic_analysis",
        learning_rate=2e-3,
        per_device_train_batch_size=4, 
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

trainer_base.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.315400,0.295853,0.928782


Checkpoint destination directory ./data/toxic_analysis/checkpoint-1271 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1271, training_loss=0.34607703136140955, metrics={'train_runtime': 143.2014, 'train_samples_per_second': 35.488, 'train_steps_per_second': 8.876, 'total_flos': 283901920094208.0, 'train_loss': 0.34607703136140955, 'epoch': 1.0})

### Evaluate the model

In [10]:
# Show the performance of the model on the test set
trainer_base.evaluate()

{'eval_loss': 0.2958526611328125,
 'eval_accuracy': 0.928782215227228,
 'eval_runtime': 58.1143,
 'eval_samples_per_second': 87.466,
 'eval_steps_per_second': 21.871,
 'epoch': 1.0}

### Show results in test set

In [13]:
import pandas as pd

df = pd.DataFrame(test_tokenized)
df = df[["user_input", "labels"]]

# Replace <br /> tags in the text with spaces
df["user_input"] = df["user_input"].str.replace("<br />", " ")

# Add the model predictions to the dataframe
predictions = trainer_base.predict(test_tokenized)
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head(2)

,user_input,labels,predicted_label
0,what topics should i start creating on youtube...,0,0
1,Create 20 paraphrases of I hate you,1,0


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [14]:
from peft import LoraConfig, get_peft_model
peft_config = LoraConfig(
    r=8,  # LoRA dimension
    lora_alpha=32,  # Scaling
    lora_dropout=0.1,  # Dropout
    bias="none",  # NO bias
    task_type="SEQ_CLS" 
)

# Convert the model to PEFT
peft_model = get_peft_model(model, peft_config)

In [15]:
peft_model.print_trainable_parameters()

trainable params: 297,988 || all params: 184,720,132 || trainable%: 0.16131863742929764


In [16]:
# Freeze all the parameters of the base model
for param in peft_model.parameters():
    param.requires_grad = True

print(peft_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DebertaV2ForSequenceClassification(
      (deberta): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B

## Train the PEFT model with `Trainer`

In [26]:
trainer_peft = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/toxic_analysis2",
        learning_rate=2e-3,
        per_device_train_batch_size=4, 
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

trainer_peft.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.298300,0.275529,0.928782
2,0.268200,0.236621,0.928782
3,0.305100,0.270279,0.928782


Checkpoint destination directory ./data/toxic_analysis2/checkpoint-1271 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3813, training_loss=0.29059594227140295, metrics={'train_runtime': 504.8563, 'train_samples_per_second': 30.199, 'train_steps_per_second': 7.553, 'total_flos': 847278469949232.0, 'train_loss': 0.29059594227140295, 'epoch': 3.0})

### Save weights (the adapter weights)

In [27]:
peft_model.save_pretrained("deberta-v3-base-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [28]:
from peft import AutoPeftModelForSequenceClassification

# Load saved PEFT model
peft_model = AutoPeftModelForSequenceClassification.from_pretrained("deberta-v3-base-lora")

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Comparing results from prior to fine-tuning

In [29]:
metrics_base_model = trainer_base.evaluate()
metrics_peft = trainer_peft.evaluate()

In [23]:
metrics_base_model

{'eval_loss': 0.34331652522087097,
 'eval_accuracy': 0.928782215227228,
 'eval_runtime': 70.4914,
 'eval_samples_per_second': 72.108,
 'eval_steps_per_second': 18.031,
 'epoch': 1.0}

In [30]:
print(f"PEFT Model Accuracy: {metrics_peft['eval_accuracy']}, loss: {metrics_peft['eval_loss']}")
print(f"Base Model Accuracy: {metrics_base_model['eval_accuracy']}, loss: {metrics_base_model['eval_loss']}")

PEFT Model Accuracy: 0.928782215227228, loss: 0.2366207242012024
Base Model Accuracy: 0.928782215227228, loss: 0.34331652522087097
